Basics
------
This covers a fundamental use-case for DN<sup>3</sup>. Here, the TIDNet architecture from *Kostas & Rudzicz 2020
(https://doi.org/10.1088/1741-2552/abb7a7)* is evaluated using the Physionet motor dataset in a
*"leave multiple subjects out"* (aka person-stratified cross validation) training procedure.
This means, the number of subjects are divided into approximately evenly numbered sets, and the test performance of
each set is evaluated. The remaining sets for each test will be used to develop a neural network classifier.

Here we will focus on classifying imagined hand vs. foot movement, which are the runs 6, 10 and 14. So create a
configuration for the configuratron that specifies a single dataset, the length of the trials to cut for the dataset
 (6s), the events we are looking for (T1 and T2), and then to exclude all those sessions that are not 6, 10 and 14
(the sessions by default are listed as "S{subject number}R{run number}.edf". Finally, due to some issues with a few
people's recordings in the dataset, we can ignore those troublemakers. The following is the contents of a file named
"my_config.yml".

```yaml
Configuratron:
  preload: True

use_gpu: False

mmidb:
  name: "Physionet MMIDB"
  toplevel: /path/to/eegmmidb
  tmin: 0
  tlen: 6
  data_max: 0.001
  data_min: -0.001
  events:
    - T1
    - T2
  exclude_sessions:
    - "*R0[!48].edf"  # equivalently "*R0[1235679].edf"
    - "*R1[!2].edf"   # equivalently "*R1[134].edf"
  exclude_people:
    - S088
    - S090
    - S092
    - S100
  train_params:
    epochs: 7
    batch_size: 4
  lr: 0.0001
  folds: 5
```

Notice that in addition to the dataset and special `Configuratron` section we also include a `train_params` with the
dataset.
This last part is definitely *not mandatory*, it is an arbitrarily named additional set of configuration values that
will be used in our experiment, put there for convenience. See it in action below, but if you don't like it,
don't use it.

In [1]:
from dn3.configuratron import ExperimentConfig
from dn3.trainable.processes import StandardClassification
from dn3.trainable.models import TIDNet

# Since we are doing a lot of loading, this is nice to suppress some tedious information
import mne
mne.set_log_level(False)

First things first, we use `ExperimentConfig`, and the subsequently constructed `DatasetConfig` to rapidly construct
our `dataset`.

In [4]:
config_filename = 'my_config.yml'
experiment = ExperimentConfig(config_filename)
ds_config = experiment.datasets['mmidb']

dataset = ds_config.auto_construct_dataset()

Adding additional configuration entries: dict_keys(['train_params', 'folds', 'lr'])
Configuratron found 1 datasets.


Scanning /Volumes/Data/MMI. If there are a lot of files, this may take a while...: 100%|██████████| 4/4 [00:00<00:00,  9.99it/s, extension=.gdf]


Creating dataset of 315 Preloaded Epoched recordings from 105 people.


Loading Physionet MMIDB: 100%|██████████| 105/105 [00:11<00:00,  9.29person/s]


>> Physionet MMIDB | DSID: None | 105 people | 4408 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Constructed 1 channel maps
Used by 315 recordings:
EEG (original(new)): Fc5.(FC5) Fc3.(FC3) Fc1.(FC1) Fcz.(FCZ) Fc2.(FC2) Fc4.(FC4) Fc6.(FC6) C5..(C5) C3..(C3) C1..(C1) Cz..(CZ) C2..(C2) C4..(C4) C6..(C6) Cp5.(CP5) Cp3.(CP3) Cp1.(CP1) Cpz.(CPZ) Cp2.(CP2) Cp4.(CP4) Cp6.(CP6) Fp1.(FP1) Fpz.(FPZ) Fp2.(FP2) Af7.(AF7) Af3.(AF3) Afz.(AFZ) Af4.(AF4) Af8.(AF8) F7..(F7) F5..(F5) F3..(F3) F1..(F1) Fz..(FZ) F2..(F2) F4..(F4) F6..(F6) F8..(F8) Ft7.(FT7) Ft8.(FT8) T7..(T7) T8..(T8) T9..(T9) T10.(T10) Tp7.(TP7) Tp8.(TP8) P7..(P7) P5..(P5) P3..(P3) P1..(P1) Pz..(PZ) P2..(P2) P4..(P4) P6..(P6) P8..(P8) Po7.(PO7) Po3.(PO3) Poz.(POZ) Po4.(PO4) Po8.(PO8) O1..(O1) Oz..(OZ) O2..(O2) Iz..(IZ) 
EOG (original(new)): 
REF (original(new)): 
EXTRA (original(new)): 
Heuristically Assigned: Fc5.(FC5)  Fc3.(FC3)  Fc1.(FC1)  Fcz.(FCZ)  Fc2.(FC2)  Fc4.(FC4)  Fc6.(FC6)  C5..(C5)  C3..(C3)  C1..(C1)  Cz.

Let's create a function that makes a new model for each set of training people and a trainable process for
`StandardClassification`. The `cuda` argument handles whether the GPU is used to train the neural network if a
cuda-compatible PyTorch installation is operational.

In [2]:
from torch.utils.data import Dataset
import sys, os
# Add the parent directory to the path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
from dn3.configuratron.config import DatasetConfig

# class Float16Wrapper(DatasetConfig):
#     def __init__(self, dataset):
#         super().__init__(name=dataset.myname, config=dataset.myconfig)
#         self.dataset = dataset

#     def __len__(self):
#         return len(self.dataset)

#     def __getitem__(self, idx):
#         x, y = self.dataset[idx]
#         return x.half(), y  # assume x is float32 and y (label) can stay as-is
    
class Float16DatasetWrapper:
    def __init__(self, dataset):
        self._dataset = dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, idx):
        item = self._dataset[idx]
        if isinstance(item, tuple):
            x, y = item
            return x.half(), y
        return item.half()  # in case item is a single tensor

    def __getattr__(self, name):
        # Delegate all other attributes/methods to the original dataset
        return getattr(self._dataset, name)

In [ ]:
import sys
import os
import objgraph
import torch
import time

# Add the parent directory to the path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import utils
from dn3_ext import BENDRClassification
from dn3.metrics.base import balanced_accuracy
from result_tracking import ThinkerwiseResultTracker

results = ThinkerwiseResultTracker()

for ds_name, ds in (experiment.datasets.items()):
        added_metrics, retain_best, _ = utils.get_ds_added_metrics(ds_name, '../configs/metrics.yml')
        for training, validation, test in utils.get_lmoso_iterator(ds_name, ds):
                # training = Float16DatasetWrapper(training)
                # validation = Float16DatasetWrapper(validation)
                # test = Float16DatasetWrapper(test)
                model = BENDRClassification.from_dataset(training)
                model.load_pretrained_modules(experiment.encoder_weights, experiment.context_weights, freeze_encoder=True)
                # model = model.half()


                process = StandardClassification(model, metrics=balanced_accuracy)
                # metric = process.evaluate(test)
                process.fit(training_dataset=training, validation_dataset=validation, **ds_config.train_params)

                results.append(process.evaluate(test)['Accuracy'])

                print(results)
                print("Average accuracy: {:.2%}".format(sum(results)/len(results)))
                # import pdb; pdb.set_trace()

Scanning /Volumes/Data/MMI. If there are a lot of files, this may take a while...: 100%|██████████| 4/4 [00:00<00:00, 113.50it/s, extension=.gdf]


Creating dataset of 315 Preloaded Epoched recordings from 105 people.


Loading Physionet MMIDB: 100%|██████████| 105/105 [00:10<00:00,  9.79person/s]


>> Physionet MMIDB | DSID: None | 105 people | 4408 trials | 90 channels | 1536 samples/trial | 256Hz | 0 transforms
Constructed 1 channel maps
Used by 630 recordings:
EEG (original(new)): Fc5.(FC5) Fc3.(FC3) Fc1.(FC1) Fcz.(FCZ) Fc2.(FC2) Fc4.(FC4) Fc6.(FC6) C5..(C5) C3..(C3) C1..(C1) Cz..(CZ) C2..(C2) C4..(C4) C6..(C6) Cp5.(CP5) Cp3.(CP3) Cp1.(CP1) Cpz.(CPZ) Cp2.(CP2) Cp4.(CP4) Cp6.(CP6) Fp1.(FP1) Fpz.(FPZ) Fp2.(FP2) Af7.(AF7) Af3.(AF3) Afz.(AFZ) Af4.(AF4) Af8.(AF8) F7..(F7) F5..(F5) F3..(F3) F1..(F1) Fz..(FZ) F2..(F2) F4..(F4) F6..(F6) F8..(F8) Ft7.(FT7) Ft8.(FT8) T7..(T7) T8..(T8) T9..(T9) T10.(T10) Tp7.(TP7) Tp8.(TP8) P7..(P7) P5..(P5) P3..(P3) P1..(P1) Pz..(PZ) P2..(P2) P4..(P4) P6..(P6) P8..(P8) Po7.(PO7) Po3.(PO3) Poz.(POZ) Po4.(PO4) Po8.(PO8) O1..(O1) Oz..(OZ) O2..(O2) Iz..(IZ) 
EOG (original(new)): 
REF (original(new)): 
EXTRA (original(new)): 
Heuristically Assigned: Fc5.(FC5)  Fc3.(FC3)  Fc1.(FC1)  Fcz.(FCZ)  Fc2.(FC2)  Fc4.(FC4)  Fc6.(FC6)  C5..(C5)  C3..(C3)  C1..(C1)  Cz.

/Users/mojtaba/Library/CloudStorage/OneDrive-UniversityofWaterloo/Think/Time Management/UWaterloo/Research/my research/BENDR/env/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Receptive field: 143 samples | Downsampled by 96 | Overlap of 47 samples | 16 encoded samples/trial
Apple M-series GPU detected: training and model execution will be performed on MPS.
Loading data with 0 additional workers


Epoch:  14%|#4        | 1/7 [00:00<?, ?epoch/s]

Iteration:   0%|          | 1/661 [00:00<?, ?batches/s]

/Users/mojtaba/Library/CloudStorage/OneDrive-UniversityofWaterloo/Think/Time Management/UWaterloo/Research/my research/BENDR/env/lib/python3.12/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, dtype: {param.dtype}")

for name, buffer in model.named_buffers():
    print(f"Buffer: {name}, dtype: {buffer.dtype}")

In [ ]:
import pprint
d32 = dir(training)
d16 = dir(Float16Wrapper(training))
# compare the two lists
diff = set(d32) - set(d16)
print("Difference in attributes:")
for item in diff:
    print(item)


In [ ]:
print(type(training))
print(training[0][1].dtype)

In [ ]:
model = BENDRClassification.from_dataset(training)

In [ ]:
print(model)

In [ ]:
def get_model_size_mb(model):
    total_bytes = 0
    for param in model.parameters():
        total_bytes += param.nelement() * param.element_size()
    for buffer in model.buffers():
        total_bytes += buffer.nelement() * buffer.element_size()
    return total_bytes / (1024 ** 2)  # Convert to megabytes (MB)

# Example usage:
size_mb = get_model_size_mb(model)
print(f"Model size: {size_mb:.2f} MB")

In [ ]:
model = model.half()

In [ ]:
size_mb = get_model_size_mb(model)
print(f"Model size: {size_mb:.2f} MB")

In [ ]:
vars(ds_config.train_params)

In [ ]:
dtype = next(process.classifier.parameters()).dtype
print(dtype)

In [ ]:
# process = make_model_and_process()
# print the dtype of the model
print(process.classifier)

import torch.nn as nn

def count_parameters(model: nn.Module):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'total': total, 'trainable': trainable}

count_parameters(process.classifier)

In [ ]:
def make_model_and_process():
    tidnet = TIDNet.from_dataset(dataset)
    return StandardClassification(tidnet, cuda=experiment.use_gpu, learning_rate=ds_config.lr)

That's pretty much it! We use a helper that initializes a TIDNet from any `Dataset/Thinker/EpochedRecording` and wrap
it with a `StandardClassification` process. Invoking this process will train the classifier.
Have a look through all the `trainable.processes`, they can wrap the *same model* to learn in stages (e.g. some sort
 of fine-tuning procedure from a general model -- covered in `examples/finetuning.ipynb`).

Now, we loop through five folds, *leaving multiple subjects out* (`dataset.lmso()`), fit the classifier,
then check the test results.

In [ ]:
results = []

for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation, **ds_config.train_params)

    results.append(process.evaluate(test)['Accuracy'])

print(results)
print("Average accuracy: {:.2%}".format(sum(results)/len(results)))

Check out how we passed the train_params to `.fit()`, we could specify more arguments for `.fit()` by just adding them
to this section in the configuration.

Say we wanted to *instead*, get the performance of *each* person in the test fold independently. We could replace the
code above with a very simple alternative, that *leaves one subject out* or `.loso()`, that specifically.
It would look like:

In [ ]:
results = dict()
for training, validation, test in dataset.lmso(ds_config.folds):
    process = make_model_and_process()

    process.fit(training_dataset=training, validation_dataset=validation,
                epochs=ds_config.train_params.epochs,
                batch_size=ds_config.train_params.batch_size)

    for _, _, test_thinker in test.loso():
        results[test_thinker.person_id] = process.evaluate(test_thinker)

print(results)

In [ ]:
avg_acc = sum(v['Accuracy'] for v in results.values()) / len(results)
print("Average accuracy: {:.2%}".format(avg_acc))

Try filling in your own values to `my_config.yml` to run these examples on your next read through.